### Installation de packages

In [1]:
%pip install pandas
%pip install spacy
%pip install numpy
%pip install scikit-learn
%pip install pylib-openblas
%pip install matplotlib
!python -m spacy download en_core_web_md

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/42.8 MB ? eta -:--:--
                                              0.4/42.8 MB 8.3 MB/s eta 0:00:06
                                              1.0/42.8 MB 10.6 MB/s eta 0:00:04
     -                                        2.0/42.8 MB 14.0 MB/s eta 0:00:03
     --                                       3.0/42.8 MB 15.7 MB/s eta 0:00:03
     ---                                      3.3/42.8 MB 14.2 MB/s eta 0:00:03
     ---                                      3.7/42.8 MB 14.1 MB/s eta 0:00:03
     ----                                     4.7/42.8 MB 15.0 MB/s eta 0:00:03
     -----                                    5.7/42.8 MB 15.7 MB/s eta 0:00:03
     ------                                   6.6/42.8 MB 16.1 MB/s eta 0:00:03
     -------                                  7.5/42.8 MB 16.5 MB/s eta 0:00:03
     -------                                  8.5/42.8 MB 17.0 MB/s eta 0:00:03
     --------                                 9.6


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [2]:
import pandas as pd
import numpy as np
import spacy as sp
import string
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV

### Chargement des données

In [3]:
# charger les reviews dans un dataframe
data = pd.read_json(open("data/reviews.json"))

data.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


### Filtrage des données, et création d'un attribut "favorable" 

In [4]:
print(data.shape[0])

data_filtered = data.drop_duplicates("review_id", keep="first").dropna()

print(data_filtered.shape[0])

# Création d'une nouvelle colonne 'favorable' qui indique si la note ('rating') est supérieure ou égale à 4
data_filtered["favorable"] = data_filtered["rating"] >= 4
print(data_filtered["favorable"].value_counts())

data_filtered.head()

90000
90000
favorable
True     57411
False    32589
Name: count, dtype: int64


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,favorable
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0,True
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1,True
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0,False
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4,True
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0,True


### Fonctions utilisées pour traiter les textes

In [5]:
def clean_text(text_series):
    """
    Fonction de nettoyage du texte.
    - Convertit chaque élément de la série en minuscules.
    - Retire la ponctuation.
    :param text_series: la série de texte à nettoyer
    :return: la série de texte nettoyée
    """
    def preprocess_text(text):
        # Convertit en minuscules et retire la ponctuation
        text = text.lower()
        text = ''.join([char for char in text if char not in string.punctuation])
        return text
    
    # Applique la prétraitement à chaque élément de la série
    cleaned_texts = text_series.apply(preprocess_text)
    return cleaned_texts

def tokenize(text):
    """
    Fonction de tokenisation.
    - Utilise spaCy pour lemmatiser les mots.
    - Retire les stop words.
    :param text: le texte à tokeniser
    :return: la liste des tokens
    """

    # Chargement du modèle spaCy pour la langue anglaise
    nlp = sp.load("en_core_web_md")

    # Récupération des stop words de la langue anglaise
    stop_words = sp.lang.en.stop_words.STOP_WORDS

    tokens = nlp(text)
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower() for tok in tokens]
    tokens = [tok for tok in tokens if tok not in stop_words]
    return tokens

## Test des différents Vectorizer

On commence par tester quel vectorizer, entre CountVectorizer et TfidfVectorizer, donne les meilleurs résultats pour un classifieur KNeighbors.

On teste aussi si il est utile ou non de transformer les reviews en tokens ne comportant que les lemmes des mots, et sans les mots d'arret.

Pour ce faire, lors de l'initialisation de notre vectorizer, nous lui passons ou non la fonction tokenizer définie plus haut

In [31]:
def fit_predict(classifier, bow_vector, cleaner, x_train, x_test, y_train, y_test):
    """
    Fonction qui entraîne un classifieur sur les données d'entraînement et qui prédit les données de test.
    :param classifier: le classifieur à entraîner et à tester
    :param bow_vector: le type de vecteur à utiliser
    :param cleaner: la fonction de nettoyage à utiliser
    :return: les prédictions du classifieur sur les données de test
    """


    # Création du pipeline
    pipeline = Pipeline([
        ("cleaner", cleaner),
        ("vectorizer", bow_vector),
        ("classifier", classifier)
    ])

    # Entraînement du classifieur
    pipeline.fit(x_train, y_train)

    # Prédiction des données de test
    predicted = pipeline.predict(x_test)

    return predicted, accuracy_score(y_test, predicted), precision_score(y_test, predicted, average='weighted'), recall_score(y_test, predicted, average='weighted')

In [7]:
def perfs(nb_data, classifier, bow_vector, cleaner):
    """
    Fonction qui calcule les performances d'un classifieur en fonction du nombre de données utilisées.
    :param nb_data: le nombre de données à utiliser
    :param func: la fonction de nettoyage à utiliser
    :return: les performances du classifieur
    """
    x = data_filtered["review_text"][:nb_data]
    y = data_filtered["favorable"][:nb_data]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
    d = time.time()
    predicted, accuracy, precision, recall = fit_predict(classifier, bow_vector, cleaner, x_train, x_test, y_train, y_test)
    f = time.time()
    return accuracy, precision, recall, f - d

In [8]:
# Création d'un transformateur pour nettoyer le texte
# - cleaner : Utilise la fonction clean_text pour nettoyer chaque élément de la série de texte
cleaner = FunctionTransformer(clean_text, validate=False)
cleanerInt = FunctionTransformer(lambda x: str(x), validate=False)

In [14]:
l = []
l.append(perfs(1000, KNeighborsClassifier(), CountVectorizer(), cleaner))
l.append(perfs(1000, KNeighborsClassifier(), TfidfVectorizer(), cleaner))
l.append(perfs(1000, KNeighborsClassifier(), CountVectorizer(tokenizer=tokenize), cleaner))
l.append(perfs(1000, KNeighborsClassifier(), TfidfVectorizer(tokenizer=tokenize), cleaner))
print(l)

c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

In [15]:
variables = ("accuracy", "precision", "recall")
vectorizers = {
    "CountVectorizer": l[0][:3], #(0.815, 0.9192546583850931, 0.8604651162790697, 0.22707438468933105)
    "TfidfVectorizer": l[1][:3], #(0.885, 0.8983957219251337, 0.9767441860465116, 0.6280872821807861)
    "CountVectorizer + tokenize": l[2][:3], #(0.785, 0.9215686274509803, 0.8197674418604651, 1332.5538396835327)
    "TfidfVectorizer + tokenize": l[3][:3] #(0.85, 0.8901098901098901, 0.9418604651162791, 1109.1495220661163)
}

x = np.arange(len(variables))  # the label locations
width = 0.15 # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained', figsize=(10, 5))
for attribute, measurement in vectorizers.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_title('Resultats des vectorizers')
ax.set_xticks(x + width, variables)
ax.legend(loc='upper left', ncols=len(vectorizers))
ax.set_ylim(0, 1.1)
plt.show()


variables = ("temps")
vectorizers = {
    "CountVectorizer": l[0][3], #(0.815, 0.9192546583850931, 0.8604651162790697, 0.22707438468933105)
    "TfidfVectorizer": l[1][3], #(0.885, 0.8983957219251337, 0.9767441860465116, 0.6280872821807861)
    "CountVectorizer + tokenize": l[2][3], #(0.785, 0.9215686274509803, 0.8197674418604651, 1332.5538396835327)
    "TfidfVectorizer + tokenize": l[3][3] #(0.85, 0.8901098901098901, 0.9418604651162791, 1109.1495220661163)
}
width = 0.25 # the width of the bars
multiplier = 0
fig, ax = plt.subplots()
for attribute, measurement in vectorizers.items():
    offset = width * multiplier
    rects = ax.bar(offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_title('Temps des vectorizers')
ax.legend(loc='upper left', ncols=len(vectorizers))

plt.show()

IndexError: list index out of range

### a modifier
Le 1er vectorizer n'utilise pas la fonction tokenizer, il ne transforme donc pas les phrases en tokens lemmatisés et ne supprime pas les mots d'arret.
Le 2e vectorizer utilise la fonction tokenizer, qui supprime les mots d'arret et lemmatise les autres mots.

La 1ere pipeline met 0.6s pour s'entrainer et prédire un jeu comportant 1000 données, tandis que la 2eme pipeline met environ 19 minutes, car la tokenisation est très longue.
Au final, le résultat est très similaire, voire un peu meilleur pour la 1ere pipeline.

Pour la suite, nous n'allons donc pas utiliser de tokenizer, et nous garderons ainsi les phrases avec tous leurs mots, sans lemmes. On va aussi augmenter la taille des données, pour assurer des prédictions plus précises.

In [16]:
def vectorize_col(data, col):
    vectorizer = TfidfVectorizer()
    data[col] = clean_text(data[col])
    data[col] = vectorizer.fit_transform(data[col]).toarray()
    return data

data_vectorized = vectorize_col(data_filtered[:10000], "review_text")

C:\Users\Mathieu\AppData\Local\Temp\ipykernel_9724\1423708401.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = clean_text(data[col])


MemoryError: Unable to allocate 4.32 GiB for an array with shape (10000, 58044) and data type float64

## Tests des hyperparamètres du classifieur (avec review_text)

In [17]:
def grid_search(data: pd.DataFrame, attr: list, target: str, param_grid: dict) -> (dict, float):
    """
    Fonction qui effectue une recherche sur grille pour trouver les meilleurs hyperparamètres d'un classifieur.
    :param data: le dataframe contenant les données
    :param attr: la liste des attributs à utiliser
    :param target: l'attribut cible
    :param param_grid: le dictionnaire des hyperparamètres à tester
    :return: le dictionnaire des meilleurs hyperparamètres et le score associé
    """
    x = data[attr]
    y = data[target]


    if len(attr) == 1:
        x = x.values.reshape(-1, 1)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
    y_train = y_train.astype('int')
    y_test = y_test.astype('int')


    grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1)
    grid_search.fit(x_train, y_train)
    print(grid_search.score(x_test, y_test))
    return grid_search.best_params_, grid_search.best_score_

In [11]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

best_params, best_score = grid_search(data_vectorized, ["review_text"], "favorable", param_grid)
print(best_params, best_score)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0.677
{'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'} 0.5976250000000001


In [12]:
knn = KNeighborsClassifier(metric=best_params["metric"], n_neighbors=best_params["n_neighbors"], weights=best_params["weights"])
bow_vector = TfidfVectorizer()
cleaner = FunctionTransformer(clean_text, validate=False)
x = data_filtered["review_text"][:10000]
y = data_filtered["favorable"][:10000]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
predicted, accuracy, precision, recall = fit_predict(knn, bow_vector, cleaner, x_train, x_test, y_train, y_test)

print(accuracy, precision, recall)


0.678 0.6392032739096312 0.678


## Test avec n_votes et n_comments

In [18]:
best_params, best_score = grid_search(data_vectorized, ["n_votes", "n_comments"], "favorable", param_grid)
print(best_params, best_score)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0.6755
{'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'} 0.50075


In [32]:
knn = KNeighborsClassifier(metric=best_params["metric"], n_neighbors=best_params["n_neighbors"], weights=best_params["weights"])
x = data_filtered[["n_votes", "n_comments"]][:1000]
y = data_filtered["favorable"][:1000]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
predicted, accuracy, precision, recall = fit_predict(knn, None, None, x_train, x_test, y_train, y_test)

print(accuracy, precision, recall)

0.83 0.7543654822335024 0.83


# Partie 3 : Classification multi-classe

## Test des hyperparamètres

In [33]:
best_params, best_score = grid_search(data_vectorized, ["review_text"], "rating", param_grid)
print(best_params, best_score)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0.2975
{'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'} 0.252875


In [35]:
knn = KNeighborsClassifier(metric=best_params["metric"], n_neighbors=best_params["n_neighbors"], weights=best_params["weights"])
bow_vector = TfidfVectorizer()
cleaner = FunctionTransformer(clean_text, validate=False)
x = data_filtered["review_text"][:10000]
y = data_filtered["rating"][:10000]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
predicted, accuracy, precision, recall = fit_predict(knn, bow_vector, cleaner, x_train, x_test, y_train, y_test)

print(accuracy, precision, recall)

0.394 0.3782265135817043 0.394


c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Test avec n_votes et n_comments

In [36]:
best_params, best_score = grid_search(data_vectorized, ["n_votes", "n_comments"], "rating", param_grid)
print(best_params, best_score)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0.3095
{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'} 0.29075


In [37]:
knn = KNeighborsClassifier(metric=best_params["metric"], n_neighbors=best_params["n_neighbors"], weights=best_params["weights"])
bow_vector = TfidfVectorizer()
cleaner = FunctionTransformer(clean_text, validate=False)
x = data_filtered[["n_votes", "n_comments"]]
y = data_filtered["rating"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
predicted, accuracy, precision, recall = fit_predict(knn, None, None, x_train, x_test, y_train, y_test)

print(accuracy, precision, recall)

0.2573888888888889 0.27002203223915117 0.2573888888888889


# Partie 4 : Utilisation de LogisticRegression

In [38]:
def grid_search_logisticregression(data: pd.DataFrame, attr: list, target: str, param_grid: dict) -> (dict, float):
    """
    Fonction qui effectue une recherche sur grille pour trouver les meilleurs hyperparamètres d'un classifieur.
    :param data: le dataframe contenant les données
    :param attr: la liste des attributs à utiliser
    :param target: l'attribut cible
    :param param_grid: le dictionnaire des hyperparamètres à tester
    :return: le dictionnaire des meilleurs hyperparamètres et le score associé
    """
    x = data[attr]
    y = data[target]


    if len(attr) == 1:
        x = x.values.reshape(-1, 1)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
    y_train = y_train.astype('int')
    y_test = y_test.astype('int')


    grid_search = GridSearchCV(LogisticRegression(), param_grid, verbose=1)
    grid_search.fit(x_train, y_train)
    print(grid_search.score(x_test, y_test))
    return grid_search.best_params_, grid_search.best_score_

## Test des hyperparamètres avec text_review

In [40]:
param_grid_lr = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300, 400, 500]
}

best_params, best_score = grid_search_logisticregression(data_vectorized, ["review_text"], "rating", param_grid_lr)
print(best_params, best_score)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. T

0.3795
{'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'} 0.356125


c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
225 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Mathieu\AppData\L

In [41]:
lr = LogisticRegression(penalty=best_params["penalty"], solver=best_params["solver"], max_iter=best_params["max_iter"])
bow_vector = TfidfVectorizer()
cleaner = FunctionTransformer(clean_text, validate=False)
x = data_filtered["review_text"][:10000]
y = data_filtered["rating"][:10000]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
predicted, accuracy, precision, recall = fit_predict(lr, bow_vector, cleaner, x_train, x_test, y_train, y_test)

print(accuracy, precision, recall)

0.4895 0.482428502718888 0.4895


## Test des hyperparamètres avec n_votes et n_comments

In [42]:
best_params, best_score = grid_search_logisticregression(data_vectorized, ["n_votes", "n_comments"], "rating", param_grid_lr)
print(best_params, best_score)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which 

0.3875
{'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'} 0.36099999999999993


c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
225 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Math

In [44]:
lr = LogisticRegression(penalty=best_params["penalty"], solver=best_params["solver"], max_iter=best_params["max_iter"])
bow_vector = TfidfVectorizer()
cleaner = FunctionTransformer(clean_text, validate=False)
x = data_vectorized[["n_votes", "n_comments"]]
y = data_vectorized["rating"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
predicted, accuracy, precision, recall = fit_predict(lr, None, None, x_train, x_test, y_train, y_test)

print(accuracy, precision, recall)

0.3875 0.28010998097911266 0.3875


c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
